In [1]:
import pyspark

import re
import ast
import time
import numpy as np
#import pandas as pd


sc = pyspark.SparkContext()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1544586234495_0002,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-1, master=yarn) created by __init__ at /tmp/1260729744655070957:589 
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 115, in __init__
    SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 316, in _ensure_initialized
    callsite.function, callsite.file, callsite.linenum))
ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-1, master=yarn) created by __init__ at /tmp/1260729744655070957:589 



In [2]:
from pyspark.sql import SQLContext

VBox()

In [3]:
trainRDD = sc.textFile('s3://breastcancerfile/e-AKVLH55FZCUE4SN1JHTY7DWU4/train.txt')

VBox()

In [4]:
TrainRDD, TrainRDD2 = trainRDD.randomSplit([0.01,0.99], seed = 2018)
TrainRDD2.unpersist()

VBox()

PythonRDD[2] at RDD at PythonRDD.scala:52

In [5]:
TrainRDD.count()

VBox()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53424)
----------------------------------------
458906
Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 263, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 238, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 251, i

In [6]:
NodeMeans = [(1.9136993727636809, 7.184627898705608),
 (105.84841979766556, 391.45781841729996),
 (21.13629851448115, 352.8574390110648),
 (5.735263227368864, 8.346464252535688),
 (18060.51214960742, 68556.28645274039),
 (90.10443053155232, 340.53335300283976),
 (15.62662976373116, 64.6908374702787),
 (12.510823839914709, 16.68706965443228),
 (101.51997332409383, 216.54476824937402),
 (0.33741472109766746, 0.59176564119444),
 (2.6146237734976343, 5.115681630473718),
 (0.2328159762771082, 2.7454850748632738),
 (6.436072751813085, 14.741644511057435)]

Needed = sc.broadcast(NodeMeans)

VBox()

In [7]:
def NumericValuesOnly(LineRDD):
    """Function takes the RDD and subsets the RDD"""
    Node = Needed.value
    Values = LineRDD.split('\t')
    ZZ = []
    for ii in range(0,40):
        
        if ii < 14:
            if Values[ii] == '':
                ZZ.append(Node[ii-1][0])
            else:
                if (ii == 0):
                    ZZ.append(float(Values[ii]))
                else:
                    ZZ.append((float(Values[ii])- float(Node[ii-1][0]))/float(Node[ii-1][1]))
        else:
            if Values[ii] == '':
                ZZ.append(str(0))
            else:
                ZZ.append(str(Values[ii]))
    
    return (((Values[14], Values[15], Values[16]), ZZ))

VBox()

In [8]:
NonNumericRDD = TrainRDD.map(NumericValuesOnly).cache()
NonNumericRDD.take(10)

VBox()

[((u'05db9164', u'80e26c9b', u'5c7d8ff6'), [1.0, 0.15119789675287548, -0.2729500211023082, 21.13629851448115, 5.735263227368864, -0.25372599727376444, -0.25285168037808703, -0.02514467005437195, -0.7497316244851707, 0.9211953181255527, 1.119675142958533, 0.6617644472509671, 1.007903502757413, 6.436072751813085, '05db9164', '80e26c9b', '5c7d8ff6', '902872c9', '384874ce', '7e0ccccf', '92ce5a7d', '0b153874', 'a73ee510', 'd1a1d478', 'e0c3cae0', '79b87c55', 'e8df3343', 'b28479f6', '4c1df281', '5627d7e0', '8efede7f', 'f54016b9', '21ddcdc9', 'b1252a9d', '4063500f', '0', '3a171ecb', '54baf4d1', 'e8b83407', '98f9ccac']), ((u'68fd1e64', u'78ccd99e', u'24ef584c'), [0.0, 1.9136993727636809, -0.27039546744939336, 21.13629851448115, -0.327715203061929, -0.234384226174274, -0.24991511046156653, -0.1488097872925819, -0.4500984292301882, -0.4457275698895711, 0.33741472109766746, -0.3156224116605392, 0.2328159762771082, -0.23308612205617563, '68fd1e64', '78ccd99e', '24ef584c', 'c8167e68', '4cf72387', '0

In [9]:
StartFromA = 14
for ii in range(14, 40):
    Node = ii
    def ValuesNonNumericFeatures(toyRDDLine):

        """ Take the node value from a broadcast variable that is sent to the function """
        Values = toyRDDLine.split('\t')
        T14 = Values[Node]
        return (T14, 1)


    def NonNumericFeatures(toyRDDLine):

        """ Take the node value from a broadcast variable that is sent to the function """
        Values = toyRDDLine.split('\t')
        T14 = Values[Node]
        T0 = Values[0]
        return (((T14, T0), 1))

    def ConvertForMerge(toyRDDLine):

        YY = toyRDDLine[0][0]
        YYY = toyRDDLine[0][1]
        YYZ = toyRDDLine[1]

        return ((YY,(YYY,YYZ)))


    def MovingOn(trainRDD):

        Key = trainRDD[0]
        Value = trainRDD[1]
        
        ValueKey = Value[0][0]
        ValueValue = Value[0][1]
        ValueDivide = Value[1]
        
        return ((Key, (ValueKey, float(ValueValue/ValueDivide))))


    def HasherFunction(trainRDD):
    
        Key = trainRDD[0]
        Value = trainRDD[1][1]
        
        if (Value >0.9):
            FinalValue = 1
        elif (Value <=0.9) & (Value >0.8):
            FinalValue = 2
        elif (Value <=0.8) & (Value >0.7):
            FinalValue = 3
        elif (Value <=0.7) & (Value>0.6):
            FinalValue = 4
        elif (Value <=0.6) & (Value >0.5):
            FinalValue = 5
        elif (Value <=0.5) & (Value >0.4):
            FinalValue = 6
        elif (Value <=0.4) & (Value >0.3):
            FinalValue = 7
        elif (Value <=0.3) & (Value >0.2):
            FinalValue =8
        elif (Value <=0.2) & (Value >0.1):
            FinalValue = 9
        elif (Value <= 0.1):
            FinalValue = 10
            
        return ((Key, FinalValue))


#def ConvertToSignificance(toyRDDLine):
#print(trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[1]))
    
    YY = TrainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).cache()

    ZZ = TrainRDD.map(NonNumericFeatures)\
                 .reduceByKey(lambda x,y: x+y)\
                 .map(ConvertForMerge)\
                 .filter(lambda x: x[1][0] == '1')\
                 .leftOuterJoin(YY)\
                 .map(MovingOn).map(HasherFunction).cache()
    TT = ZZ.collect()

    ZZ.unpersist()
    YY.unpersist()

    #NamingDict = "HashDictionary" + str(Node)
    from collections import defaultdict

    NamingDict = defaultdict(list)

    for ii in TT:
        NamingDict[ii[0]] = ii[1]
        

        
    YYY = sc.broadcast(NamingDict)
    def MappingChangesWithDictionary(trainRDD):


        """ Mapping Changes with What the Dictionary Has """
        Dictionary = YYY.value

        """ Taking in all the Key/Value Components """
        FinalKey = trainRDD[0]
        


        Value = trainRDD[1]
        Value[Node] = Dictionary.get(Value[Node], 0)

        return (FinalKey,Value)



    NonNumericRDD = NonNumericRDD.map(MappingChangesWithDictionary).cache()
    
    
NonNumericRDD.take(1)

VBox()

[((u'05db9164', u'80e26c9b', u'5c7d8ff6'), [1.0, 0.15119789675287548, -0.2729500211023082, 21.13629851448115, 5.735263227368864, -0.25372599727376444, -0.25285168037808703, -0.02514467005437195, -0.7497316244851707, 0.9211953181255527, 1.119675142958533, 0.6617644472509671, 1.007903502757413, 6.436072751813085, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 10, 10, 10, 10])]

In [11]:
def sigmaFunction(z):
    yhat = 1.0/(1.0 + np.exp(-1 * z))
    return (yhat)

VBox()

In [12]:
initialWeights = (np.ones(300))

VBox()

In [32]:
lrate = 0.7
initialWeights = list(np.ones(300))
def predict(RDDRow):
    
    """ The first Coefficient is always the intercept or the Bias term"""
    Key = RDDRow[0]
    Value = RDDRow[1]
    realY = Value[0]
    RestY = Value[1:14]
    RestYCat = Value[14:]
    #RestY.insert(1,0)
    initialWeights = initWeight.value
    Y = initialWeights[0]
    
    for i in range(len(RestY)-1):
        
        Y+= initialWeights[i+1] *RestY[i]
    
    for index,value in enumerate(RestYCat):
        Y += initialWeights[14+(value)+(index*11)]
    
        
    ZZ = sigmaFunction(Y)
    
    Error = (realY - ZZ)
    ErrorSquared = Error**2
    
    
    for index,value in enumerate(initialWeights):

        if index == 0:
            initialWeights[index] = initialWeights[index] + (lrate * Error * ZZ * (1.0 - ZZ))
        else:
            
            if index <= 14:
                initialWeights[index] = initialWeights[index] + ((lrate* Error * ZZ * (1.0 - ZZ))*Value[index-1])
    
    for indices, values in enumerate(RestYCat):
        
        
        IndexToUpdate = 14+values +(indices*11)
        initialWeights[IndexToUpdate] = initialWeights[IndexToUpdate] + ((lrate* Error * ZZ * (1.0 - ZZ)))
                
                                                

    
    
    return(initialWeights)

VBox()

In [14]:
PredictionCalc = NonNumericRDD.map(predict).cache()
Y = PredictionCalc.collect()[-1]
Y

VBox()

[0.9999999995924044, 1.0, 1.000000000062389, 1.000000000097784, 1.0000000000097127, 1.0000000001410299, 1.000000000073139, 1.000000000029065, 1.0000000000740206, 1.000000000192697, 1.000000000099937, 1.0000000001310472, 1.0000000001001772, 0.9999999999075673, 1.0000000000276408, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999995924044, 0.9999999999879455, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999996044588, 0.9999999997636152, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999998287892, 0.9999999999191619, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999996732425, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999995924044, 0.9999999999035522, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999996888522, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999995924044, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999995924044, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999995924044, 0.9999999999547172, 1.0, 1.0, 1.0, 

In [22]:
lrate = 0.7
initialWeights = Y
PredictionCalc = NonNumericRDD.map(predict).cache()

Y = PredictionCalc.collect()[-1]
Y

VBox()

[0.9999999991848088, 1.0, 1.0000000001247777, 1.000000000195568, 1.0000000000194251, 1.0000000002820597, 1.000000000146278, 1.00000000005813, 1.0000000001480411, 1.000000000385394, 1.000000000199874, 1.000000000262094, 1.0000000002003544, 0.9999999998151345, 1.000000000055281, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999991848088, 0.9999999999758911, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999992089177, 0.9999999995272304, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999996575784, 0.9999999998383238, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999999999346485, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999991848088, 0.9999999998071043, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999993777045, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999991848088, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999991848088, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999991848088, 0.9999999999094344, 1.0, 1.0, 1.0, 1.0

In [23]:
lrate = 0.9
initialWeights = Y
PredictionCalc = NonNumericRDD.map(predict).cache()

Y = PredictionCalc.collect()[-1]
Y

VBox()

[0.9999999987772131, 1.0, 1.0000000001871665, 1.000000000293352, 1.0000000000291376, 1.0000000004230896, 1.0000000002194172, 1.000000000087195, 1.0000000002220617, 1.000000000578091, 1.0000000002998108, 1.0000000003931406, 1.0000000003005316, 0.9999999997227018, 1.000000000082921, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999987772131, 0.9999999999638366, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999988133765, 0.9999999992908456, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999994863675, 0.9999999997574857, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999990197275, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999987772131, 0.9999999997106565, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999990665567, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999987772131, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999987772131, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999987772131, 0.9999999998641517, 1.0, 1.0, 1.0

In [33]:
def realLoss(toyRDD):
    """ The first Coefficient is always the intercept or the Bias term"""
    Key = toyRDD[0]
    Value = toyRDD[1]
    realY = Value[0]
    RestY = Value[1:14]
    RestYCat = Value[14:]
    #RestY.insert(1,0)
    
    initialWeights = initWeight.value
    Y = initialWeights[0]
    
    for i in range(len(RestY)-1):
        
        Y+= initialWeights[i+1] *RestY[i]
    
    for index,value in enumerate(RestYCat):
        Y += initialWeights[14+(value)+(index*11)]
    
        
    ZZ = sigmaFunction(Y)
    
    Error = (realY - ZZ)
    return Error

VBox()

In [ ]:
LossShift = []
initWeights = list(np.ones(300))
initWeight = sc.broadcast(initWeights)

for ii in range(40):
    lrate = 0.9
    PredictionCalc = NonNumericRDD.map(predict).cache()
    Y = PredictionCalc.collect()[-1]
    print("This is Run: ", ii)
    Y
    initWeight = sc.broadcast(Y)
    Looser = NonNumericRDD.map(realLoss).mean()
    LossShift.append(Looser)

VBox()

In [31]:
LossShift

VBox()

[-0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929, -0.7438821893789929]

In [20]:
def RunningEpochs(Epoch, lrate, initialWeights):
    Epoch = Epoch
    lrate = lrate

########################################################################
#################### Initial Start of the Function #####################

    def predict(RDDRow):
    
        """ The first Coefficient is always the intercept or the Bias term"""
        Key = RDDRow[0]
        Value = RDDRow[1]
        realY = Value[0]
        RestY = Value[1:14]
        RestYCat = Value[14:]
        #RestY.insert(1,0)

        Y = initialWeights[0]

        for i in range(len(RestY)-1):

            Y+= initialWeights[i+1] *RestY[i]

        for index,value in enumerate(RestYCat):
            Y += initialWeights[14+(value)+(index*11)]


        ZZ = sigmaFunction(Y)

        Error = (realY - ZZ)
        ErrorSquared = Error**2


        for index,value in enumerate(initialWeights):

            if index == 0:
                initialWeights[index] = initialWeights[index] + (lrate * Error * ZZ * (1.0 - ZZ))
            else:

                if index <= 14:
                    initialWeights[index] = initialWeights[index] + ((lrate* Error * ZZ * (1.0 - ZZ))*Value[index-1])

        for indices, values in enumerate(RestYCat):

            IndexToUpdate = 14+values +(indices*11)
            initialWeights[IndexToUpdate] = initialWeights[IndexToUpdate] + ((lrate* Error * ZZ * (1.0 - ZZ)))
            
        return(initialWeights)
    
################################################################################ 
##################### Final End of The Function ################################

    for ii in range(Epoch):
        
        
        PredictnewWeight= NonNumericRDD.map(predict).collect()[-1]
        initialWeights = PredictionCalc
    return initialWeights  



VBox()

In [21]:
YY = list(np.ones(300))
RunningEpochs(20, 0.89, YY)

VBox()

Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.
Traceback (most recent call last):
  File "<stdin>", line 56, in RunningEpochs
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 814, in collect
    sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 2472, in _jrdd
    self._jrdd_deserializer, profiler)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 2405, in _wrap_function
    pickled_command, broadcast_vars, env, includes = _prepare_for_python_RDD(sc, 

In [10]:
def MakingRDDDataFrames(rowRDD):
    
    """Getting the RDD in the format of Sets and Lists"""
    Key = rowRDD[0]
    ####################
    String1 = str(Key[0])
    String2 = str(Key[1])
    String3 = str(Key[2])
    FinalKey = String1 + String2 + String3
    
    Values = rowRDD[1]
    
    ### Getting each individual Value WoW
    Num1 = Values[0]
    Num2 = Values[1]
    Num3 = Values[3]
    Num4 = Values[4]
    Num5 = Values[5]
    Num6 = Values[6]
    Num7 = Values[7]
    Num8 = Values[8]
    Num9 = Values[9]
    Num10 = Values[10]
    Num11 = Values[11]
    Num12 = Values[12]
    Num13 = Values[13]
    
    return(FinalKey, Num1, Num2, Num3, Num4, Num5, Num6, Num7, Num8, Num9, Num10, Num11, Num12, Num13)

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.44.68:18888/sessions/2 with error payload: "Session '2' not found."


## Working with a random split 70:30 Toy Example -- The best splitting criterion

In [8]:
NumericRDD = TrainRDD.map(NumericValuesOnly).map(MakingRDDDataFrames).cache()
NumericRDD.take(10)

VBox()

name 'MakingRDDDataFrames' is not defined
Traceback (most recent call last):
NameError: name 'MakingRDDDataFrames' is not defined



In [10]:
NumericRDD1 = TrainRDD.map(NumericValuesOnly).cache()
NumericRDD1.take(10)

VBox()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47956)
----------------------------------------
[('68fd1e6480e26c9bfb936136', [0.0, -0.1271742093878368, -0.26784091379647845, -0.04573036226671461, -0.6871488397768515, -0.24328202434221455, -0.25285168037808703, -0.009686530399595706, -0.6298783463831777, 0.36703739055185386, 1.119675142958533, -0.12014503987823798, 0, -0.3009211589986225]), ('68fd1e64f0cf00246f67f7e5', [0.0, 0.01201184368251934, -0.27039546744939336, 0.06479586075781131, -0.567337627538544, -0.261952813940516, -0.24110540071200498, -0.21064234591168687, -0.6298783463831777, -0.4503455526193519, 1.119675142958533, -0.3156224116605392, 0, -0.16525108511372877]), ('8cf07265ae46a29dc81688bb', [0.0, 0.15119789675287548, -0.2729500211023082, 0, -0.6871488397768515, -0.2634114693778833, -0.2645979600441691, -0.19518420625691063, -0.7497316244851707, -0.4688174835384752, 1.119675142958533, -0.3156224116605392, 0, -0.4

In [9]:
from pyspark.sql import Row
from pyspark.sql.types import *

schema = StructType([StructField("Keys", StringType(), False),
                    StructField("Num1", DoubleType(), False),
                    StructField("Num2", DoubleType(), False),
                    StructField("Num3", DoubleType(), False),
                    StructField("Num4", DoubleType(), False),
                    StructField("Num5", DoubleType(), False),
                    StructField("Num6", DoubleType(), False),
                    StructField("Num7", DoubleType(), False),
                    StructField("Num8", DoubleType(), False),
                    StructField("Num9", DoubleType(), False),
                    StructField("Num10", DoubleType(), False),
                    StructField("Num11", DoubleType(), False),
                    StructField("Num12", DoubleType(), False),
                    StructField("Num13", DoubleType(), False),])

VBox()

In [13]:
Numeric = NumericRDD.map(lambda y: Row(Keys = y[0], Num1 = float(y[1]), Num2 = float(y[2]), Num3 = float(y[3]),
                                   Num4 = float(y[4]), Num5 = float(y[5]), Num6 = float(y[6]), Num7 = float(y[7]),
                                   Num8 = float(y[8]), Num9 = float(y[9]), Num10 = float(y[10]), Num11 = float(y[11]),
                                   Num12 = float(y[12]), Num13 = float(y[13])))

VBox()

In [14]:
DataFrame = spark.createDataFrame(Numeric, schema)

VBox()

u'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 693, in createDataFrame
    jdf = self._jsparkSession.applySchemaToPythonRDD(jrdd.rdd(), schema.json())
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'



In [ ]:
DataFrame.head(20)

In [11]:
def NonNumericValuesOnly(LineRDD):
    """Function takes the RDD and subsets the RDD"""
    Values = LineRDD.split('\t')
    ZZ = []
    RealKey = str(Values[14]) + str(Values[15]) + str(Values[16])
    for ii in range(14,40):
        if Values[ii] == '':
            ZZ.append(str(0))
        else:
            ZZ.append(str(Values[ii]))
    
    return (RealKey, ZZ)


def MakingRDDDataFrames(rowRDD):
    
    """Getting the RDD in the format of Sets and Lists"""
    Key = rowRDD[0]
    ####################
    
    Values = rowRDD[1]
    
    ### Getting each individual Value WoW
    Str14 = Values[0]
    Str15 = Values[1]
    Str16 = Values[3]
    Str17 = Values[4]
    Str18 = Values[5]
    Str19 = Values[6]
    Str20 = Values[7]
    Str21 = Values[8]
    Str22 = Values[9]
    Str23 = Values[10]
    Str24 = Values[11]
    Str25 = Values[12]
    Str26 = Values[13]
    Str27 = Values[14]
    Str28 = Values[15]
    Str29 = Values[16]
    Str30 = Values[17]
    Str31 = Values[18]
    Str32 = Values[19]
    Str33 = Values[20]
    Str34 = Values[21]
    Str35 = Values[22]
    Str36 = Values[23]
    Str37 = Values[24]
    Str38 = Values[25]
    Str39 = Values[26]
    
    return(FinalKey, Str14, Str15, Str16, Str17, Str18, Str19, Str20, Str21, Str22, Str23, Str24, Str25, Str26, Str27, Str28, Str29,
          Str30, Str31, Str32, Str33, Str34, Str35, Str36, Str37, Str38, Str39)

VBox()

In [12]:
NonNumericRDD = TrainRDD.map(NonNumericValuesOnly).cache()
NonNumericRDD.take(20)


VBox()

[('68fd1e6480e26c9bfb936136', ['68fd1e64', '80e26c9b', 'fb936136', '7b4723c4', '25c83c98', '7e0ccccf', 'de7995b8', '1f89b562', 'a73ee510', 'a8cd5504', 'b2cb9c98', '37c9c164', '2824a5f6', '1adce6ef', '8ba8b39a', '891b62e7', 'e5ba7672', 'f54016b9', '21ddcdc9', 'b1252a9d', '07b5194c', '0', '3a171ecb', 'c5c50484', 'e8b83407', '9727dd16']), ('68fd1e64f0cf00246f67f7e5', ['68fd1e64', 'f0cf0024', '6f67f7e5', '41274cd7', '25c83c98', 'fe6b92e5', '922afcc0', '0b153874', 'a73ee510', '2b53e5fb', '4f1b46f3', '623049e6', 'd7020589', 'b28479f6', 'e6c5b5cd', 'c92f3b61', '07c540c4', 'b04e4670', '21ddcdc9', '5840adea', '60f6221e', '0', '3a171ecb', '43f13e8b', 'e8b83407', '731c3655']), ('8cf07265ae46a29dc81688bb', ['8cf07265', 'ae46a29d', 'c81688bb', 'f922efad', '25c83c98', '13718bbd', 'ad9fa255', '0b153874', 'a73ee510', '5282c137', 'e5d8af57', '66a76a26', 'f06c53ac', '1adce6ef', '8ff4b403', '01adbab4', '1e88c74f', '26b3c7a7', '0', '0', '21c9516a', '0', '32c7478e', 'b34f3128', '0', '0']), ('05db91646c9c9c

In [16]:
##### 16

""" Nodes and their meanings """
Node = 16
def ValuesNonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    return (T14, 1)


def NonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    T0 = Values[0]
    return (((T14, T0), 1))

def ConvertForMerge(toyRDDLine):
    
    YY = toyRDDLine[0][0]
    YYY = toyRDDLine[0][1]
    YYZ = toyRDDLine[1]
    
    return ((YY,(YYY,YYZ)))

def GetImportance(toyRDDLine):
    """Get the ToyRDD and then divide by the count of occurence"""
    PayLoad = toyRDDLine
    Key = toyRDDLine[0]
    PayLoads = PayLoad[1]
    PayLoads2 = PayLoads[0]  ##This is the real payload to divide through
    Value = float(PayLoads[1])   ##This is the numerical estimate value
    
    if type(PayLoads2) == list:
        Lister= []
        for ii in PayLoads2:
            if (ii[0] == "1"):
                if (ii[1]/float(Value) >0.9):
                    Lister.append(1)
                elif (ii[1]/float(Value) <=0.9) & (ii[1]/float(Value) >0.8):
                    Lister.append(2)
                elif (ii[1]/float(Value) <=0.8) & (ii[1]/float(Value) >0.7):
                    Lister.append(3)
                elif (ii[1]/float(Value) <=0.7) & (ii[1]/float(Value) >0.6):
                    Lister.append(4)
                elif (ii[1]/float(Value) <=0.6) & (ii[1]/float(Value) >0.5):
                    Lister.append(5)
                elif (ii[1]/float(Value) <=0.5) & (ii[1]/float(Value) >0.4):
                    Lister.append(6)
                elif (ii[1]/float(Value) <=0.4) & (ii[1]/float(Value) >0.3):
                    Lister.append(7)
                elif (ii[1]/float(Value) <=0.3) & (ii[1]/float(Value) >0.2):
                    Lister.append(8)
                elif (ii[1]/float(Value) <=0.2) & (ii[1]/float(Value) >0.1):
                    Lister.append(9)
                elif (ii[1] <= 0.1):
                    Lister.append(10)
            else:
                continue
                
                
    else:
        Lister =[]
        if (PayLoads2[0] == "1"):
            if (PayLoads2[1]/float(Value) >0.9):
                Lister.append(1)
            elif (PayLoads2[1]/float(Value) <=0.9) & (PayLoads2[1]/float(Value) >0.8):
                Lister.append(2)
            elif (PayLoads2[1]/float(Value) <=0.8) & (PayLoads2[1]/float(Value) >0.7):
                Lister.append(3)
            elif (PayLoads2[1]/float(Value) <=0.7) & (PayLoads2[1]/float(Value) >0.6):
                Lister.append(4)
            elif (PayLoads2[1]/float(Value) <=0.6) & (PayLoads2[1]/float(Value) >0.5):
                Lister.append(5)
            elif (PayLoads2[1]/float(Value) <=0.5) & (PayLoads2[1]/float(Value) >0.4):
                Lister.append(6)
            elif (PayLoads2[1]/float(Value) <=0.4) & (PayLoads2[1]/float(Value) >0.3):
                Lister.append(7)
            elif (PayLoads2[1]/float(Value) <=0.3) & (PayLoads2[1]/float(Value) >0.2):
                Lister.append(8)
            elif (PayLoads2[1]/float(Value) <=0.2) & (PayLoads2[1]/float(Value) >0.1):
                Lister.append(9)
            elif (PayLoads2[1] <= 0.1):
                Lister.append(10)
        
    return (Key, Lister)


    
    


#def ConvertToSignificance(toyRDDLine):
    

#print(trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[1]))
    
YY = trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).cache()

ZZ = trainRDD.map(NonNumericFeatures)\
             .reduceByKey(lambda x,y: x+y)\
             .map(ConvertForMerge)\
             .union(YY)\
             .reduceByKey(lambda x,y: [x] + [y])\
             .map(GetImportance).filter(lambda x: len(x[1]) != 0 ).cache()
TT = ZZ.collect()

ZZ.unpersist()
YY.unpersist()

#NamingDict = "HashDictionary" + str(Node)
from collections import defaultdict

NamingDict = defaultdict(list)

for ii in TT:
    NamingDict[ii[0]] = ii[1]

VBox()

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 29.0 failed 4 times, most recent failure: Lost task 1.3 in stage 29.0 (TID 3616, ip-172-31-13-1.us-west-2.compute.internal, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000002/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000002/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000002/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.is

In [17]:
Node = 2
YYY = sc.broadcast(NamingDict)
def MappingChangesWithDictionary(trainRDD):
    
    
    Dictionary = YYY.value
    
    """ Taking in all the Key/Value Components """
    FinalKey = trainRDD[0]
    
    
    
    Value = trainRDD[1]
    Value[Node] = Dictionary[Value[Node]]
    
    return (FinalKey,Value[Node])


Hashing_1 = NonNumericRDD.map(MappingChangesWithDictionary).cache()
Hashing_1.sample(False, 0.0000001, None).collect()

VBox()

name 'NamingDict' is not defined
Traceback (most recent call last):
NameError: name 'NamingDict' is not defined



## 16

In [18]:
##### 16

""" Nodes and their meanings """
Node = 17
def ValuesNonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    return (T14, 1)


def NonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    T0 = Values[0]
    return (((T14, T0), 1))

def ConvertForMerge(toyRDDLine):
    
    YY = toyRDDLine[0][0]
    YYY = toyRDDLine[0][1]
    YYZ = toyRDDLine[1]
    
    return ((YY,(YYY,YYZ)))

def GetImportance(toyRDDLine):
    """Get the ToyRDD and then divide by the count of occurence"""
    PayLoad = toyRDDLine
    Key = toyRDDLine[0]
    PayLoads = PayLoad[1]
    PayLoads2 = PayLoads[0]  ##This is the real payload to divide through
    Value = float(PayLoads[1])   ##This is the numerical estimate value
    
    if type(PayLoads2) == list:
        Lister= []
        for ii in PayLoads2:
            if (ii[0] == "1"):
                if (ii[1]/float(Value) >0.9):
                    Lister.append(1)
                elif (ii[1]/float(Value) <=0.9) & (ii[1]/float(Value) >0.8):
                    Lister.append(2)
                elif (ii[1]/float(Value) <=0.8) & (ii[1]/float(Value) >0.7):
                    Lister.append(3)
                elif (ii[1]/float(Value) <=0.7) & (ii[1]/float(Value) >0.6):
                    Lister.append(4)
                elif (ii[1]/float(Value) <=0.6) & (ii[1]/float(Value) >0.5):
                    Lister.append(5)
                elif (ii[1]/float(Value) <=0.5) & (ii[1]/float(Value) >0.4):
                    Lister.append(6)
                elif (ii[1]/float(Value) <=0.4) & (ii[1]/float(Value) >0.3):
                    Lister.append(7)
                elif (ii[1]/float(Value) <=0.3) & (ii[1]/float(Value) >0.2):
                    Lister.append(8)
                elif (ii[1]/float(Value) <=0.2) & (ii[1]/float(Value) >0.1):
                    Lister.append(9)
                elif (ii[1] <= 0.1):
                    Lister.append(10)
            else:
                continue
                
                
    else:
        Lister =[]
        if (PayLoads2[0] == "1"):
            if (PayLoads2[1]/float(Value) >0.9):
                Lister.append(1)
            elif (PayLoads2[1]/float(Value) <=0.9) & (PayLoads2[1]/float(Value) >0.8):
                Lister.append(2)
            elif (PayLoads2[1]/float(Value) <=0.8) & (PayLoads2[1]/float(Value) >0.7):
                Lister.append(3)
            elif (PayLoads2[1]/float(Value) <=0.7) & (PayLoads2[1]/float(Value) >0.6):
                Lister.append(4)
            elif (PayLoads2[1]/float(Value) <=0.6) & (PayLoads2[1]/float(Value) >0.5):
                Lister.append(5)
            elif (PayLoads2[1]/float(Value) <=0.5) & (PayLoads2[1]/float(Value) >0.4):
                Lister.append(6)
            elif (PayLoads2[1]/float(Value) <=0.4) & (PayLoads2[1]/float(Value) >0.3):
                Lister.append(7)
            elif (PayLoads2[1]/float(Value) <=0.3) & (PayLoads2[1]/float(Value) >0.2):
                Lister.append(8)
            elif (PayLoads2[1]/float(Value) <=0.2) & (PayLoads2[1]/float(Value) >0.1):
                Lister.append(9)
            elif (PayLoads2[1] <= 0.1):
                Lister.append(10)
        
    return (Key, Lister)


    
    


#def ConvertToSignificance(toyRDDLine):
    

#print(trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[1]))
    
YY = trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).cache()

ZZ = trainRDD.map(NonNumericFeatures)\
             .reduceByKey(lambda x,y: x+y)\
             .map(ConvertForMerge)\
             .union(YY)\
             .reduceByKey(lambda x,y: [x] + [y])\
             .map(GetImportance).filter(lambda x: len(x[1]) != 0 ).cache()
TT = ZZ.collect()

ZZ.unpersist()
YY.unpersist()

#NamingDict = "HashDictionary" + str(Node)
from collections import defaultdict

NamingDict1 = defaultdict(list)

for ii in TT:
    NamingDict1[ii[0]] = ii[1]

VBox()

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 4 times, most recent failure: Lost task 0.3 in stage 38.0 (TID 4804, ip-172-31-2-59.us-west-2.compute.internal, executor 8): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000020/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000020/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt1/yarn/usercache/livy/appcache/application_1544211040330_0001/container_1544211040330_0001_01_000020/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.is

In [19]:
YYY.unpersist()

VBox()

name 'YYY' is not defined
Traceback (most recent call last):
NameError: name 'YYY' is not defined



In [20]:
Node = 3
YYY = sc.broadcast(NamingDict1)
def MappingChangesWithDictionary(trainRDD):
    
    
    Dictionary = YYY.value
    
    """ Taking in all the Key/Value Components """
    FinalKey = trainRDD[0]
    
    
    
    Value = trainRDD[1]
    Value[Node] = Dictionary[Value[Node]]
    
    return (FinalKey,Value[Node])


Hashing_2 = NonNumericRDD.map(MappingChangesWithDictionary).cache()
Hashing_2.sample(False, 0.0000001, None).collect()

VBox()

name 'NamingDict1' is not defined
Traceback (most recent call last):
NameError: name 'NamingDict1' is not defined



In [21]:
Hashing3 = Hashing_1.leftOuterJoin(Hashing_2).cache()
Hashing3.take(20)

VBox()

name 'Hashing_1' is not defined
Traceback (most recent call last):
NameError: name 'Hashing_1' is not defined



## 19

In [ ]:
##### 19

""" Nodes and their meanings """
Node = 19
def ValuesNonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    return (T14, 1)


def NonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    T0 = Values[0]
    return (((T14, T0), 1))

def ConvertForMerge(toyRDDLine):
    
    YY = toyRDDLine[0][0]
    YYY = toyRDDLine[0][1]
    YYZ = toyRDDLine[1]
    
    return ((YY,(YYY,YYZ)))

def GetImportance(toyRDDLine):
    """Get the ToyRDD and then divide by the count of occurence"""
    PayLoad = toyRDDLine
    Key = toyRDDLine[0]
    PayLoads = PayLoad[1]
    PayLoads2 = PayLoads[0]  ##This is the real payload to divide through
    Value = float(PayLoads[1])   ##This is the numerical estimate value
    
    if type(PayLoads2) == list:
        Lister= []
        for ii in PayLoads2:
            if (ii[0] == "1"):
                if (ii[1]/float(Value) >0.9):
                    Lister.append(1)
                elif (ii[1]/float(Value) <=0.9) & (ii[1]/float(Value) >0.8):
                    Lister.append(2)
                elif (ii[1]/float(Value) <=0.8) & (ii[1]/float(Value) >0.7):
                    Lister.append(3)
                elif (ii[1]/float(Value) <=0.7) & (ii[1]/float(Value) >0.6):
                    Lister.append(4)
                elif (ii[1]/float(Value) <=0.6) & (ii[1]/float(Value) >0.5):
                    Lister.append(5)
                elif (ii[1]/float(Value) <=0.5) & (ii[1]/float(Value) >0.4):
                    Lister.append(6)
                elif (ii[1]/float(Value) <=0.4) & (ii[1]/float(Value) >0.3):
                    Lister.append(7)
                elif (ii[1]/float(Value) <=0.3) & (ii[1]/float(Value) >0.2):
                    Lister.append(8)
                elif (ii[1]/float(Value) <=0.2) & (ii[1]/float(Value) >0.1):
                    Lister.append(9)
                elif (ii[1] <= 0.1):
                    Lister.append(10)
            else:
                continue
                
                
    else:
        Lister =[]
        if (PayLoads2[0] == "1"):
            if (PayLoads2[1]/float(Value) >0.9):
                Lister.append(1)
            elif (PayLoads2[1]/float(Value) <=0.9) & (PayLoads2[1]/float(Value) >0.8):
                Lister.append(2)
            elif (PayLoads2[1]/float(Value) <=0.8) & (PayLoads2[1]/float(Value) >0.7):
                Lister.append(3)
            elif (PayLoads2[1]/float(Value) <=0.7) & (PayLoads2[1]/float(Value) >0.6):
                Lister.append(4)
            elif (PayLoads2[1]/float(Value) <=0.6) & (PayLoads2[1]/float(Value) >0.5):
                Lister.append(5)
            elif (PayLoads2[1]/float(Value) <=0.5) & (PayLoads2[1]/float(Value) >0.4):
                Lister.append(6)
            elif (PayLoads2[1]/float(Value) <=0.4) & (PayLoads2[1]/float(Value) >0.3):
                Lister.append(7)
            elif (PayLoads2[1]/float(Value) <=0.3) & (PayLoads2[1]/float(Value) >0.2):
                Lister.append(8)
            elif (PayLoads2[1]/float(Value) <=0.2) & (PayLoads2[1]/float(Value) >0.1):
                Lister.append(9)
            elif (PayLoads2[1] <= 0.1):
                Lister.append(10)
        
    return (Key, Lister)


    
    


#def ConvertToSignificance(toyRDDLine):
    

#print(trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[1]))
    
YY = trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).cache()

ZZ = trainRDD.map(NonNumericFeatures)\
             .reduceByKey(lambda x,y: x+y)\
             .map(ConvertForMerge)\
             .union(YY)\
             .reduceByKey(lambda x,y: [x] + [y])\
             .map(GetImportance).filter(lambda x: len(x[1]) != 0 ).cache()
TT = ZZ.collect()

ZZ.unpersist()
YY.unpersist()

#NamingDict = "HashDictionary" + str(Node)
from collections import defaultdict

NamingDict2 = defaultdict(list)

for ii in TT:
    NamingDict2[ii[0]] = ii[1]

In [ ]:
YYY.unpersist()

In [ ]:
Node = 5
YYY = sc.broadcast(NamingDict2)
def MappingChangesWithDictionary(trainRDD):
    
    
    Dictionary = YYY.value
    
    """ Taking in all the Key/Value Components """
    FinalKey = trainRDD[0]
    
    
    
    Value = trainRDD[1]
    Value[Node] = Dictionary[Value[Node]]
    
    return (FinalKey,Value[Node])


Hashing_4 = NonNumericRDD.map(MappingChangesWithDictionary).cache()
Hashing_4.sample(False, 0.0000001, None).collect()

In [ ]:
Hashing_1.unpersist()
Hashing_2.unpersist()

In [ ]:
Hashing5 = Hashing_3.leftOuterJoin(Hashing_4).cache()
Hashing5.take(20)

## 20

In [ ]:
##### 20

""" Nodes and their meanings """
Node = 20
def ValuesNonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    return (T14, 1)


def NonNumericFeatures(toyRDDLine):

    """ Take the node value from a broadcast variable that is sent to the function """
    Values = toyRDDLine.split('\t')
    T14 = Values[Node]
    T0 = Values[0]
    return (((T14, T0), 1))

def ConvertForMerge(toyRDDLine):
    
    YY = toyRDDLine[0][0]
    YYY = toyRDDLine[0][1]
    YYZ = toyRDDLine[1]
    
    return ((YY,(YYY,YYZ)))

def GetImportance(toyRDDLine):
    """Get the ToyRDD and then divide by the count of occurence"""
    PayLoad = toyRDDLine
    Key = toyRDDLine[0]
    PayLoads = PayLoad[1]
    PayLoads2 = PayLoads[0]  ##This is the real payload to divide through
    Value = float(PayLoads[1])   ##This is the numerical estimate value
    
    if type(PayLoads2) == list:
        Lister= []
        for ii in PayLoads2:
            if (ii[0] == "1"):
                if (ii[1]/float(Value) >0.9):
                    Lister.append(1)
                elif (ii[1]/float(Value) <=0.9) & (ii[1]/float(Value) >0.8):
                    Lister.append(2)
                elif (ii[1]/float(Value) <=0.8) & (ii[1]/float(Value) >0.7):
                    Lister.append(3)
                elif (ii[1]/float(Value) <=0.7) & (ii[1]/float(Value) >0.6):
                    Lister.append(4)
                elif (ii[1]/float(Value) <=0.6) & (ii[1]/float(Value) >0.5):
                    Lister.append(5)
                elif (ii[1]/float(Value) <=0.5) & (ii[1]/float(Value) >0.4):
                    Lister.append(6)
                elif (ii[1]/float(Value) <=0.4) & (ii[1]/float(Value) >0.3):
                    Lister.append(7)
                elif (ii[1]/float(Value) <=0.3) & (ii[1]/float(Value) >0.2):
                    Lister.append(8)
                elif (ii[1]/float(Value) <=0.2) & (ii[1]/float(Value) >0.1):
                    Lister.append(9)
                elif (ii[1] <= 0.1):
                    Lister.append(10)
            else:
                continue
                
                
    else:
        Lister =[]
        if (PayLoads2[0] == "1"):
            if (PayLoads2[1]/float(Value) >0.9):
                Lister.append(1)
            elif (PayLoads2[1]/float(Value) <=0.9) & (PayLoads2[1]/float(Value) >0.8):
                Lister.append(2)
            elif (PayLoads2[1]/float(Value) <=0.8) & (PayLoads2[1]/float(Value) >0.7):
                Lister.append(3)
            elif (PayLoads2[1]/float(Value) <=0.7) & (PayLoads2[1]/float(Value) >0.6):
                Lister.append(4)
            elif (PayLoads2[1]/float(Value) <=0.6) & (PayLoads2[1]/float(Value) >0.5):
                Lister.append(5)
            elif (PayLoads2[1]/float(Value) <=0.5) & (PayLoads2[1]/float(Value) >0.4):
                Lister.append(6)
            elif (PayLoads2[1]/float(Value) <=0.4) & (PayLoads2[1]/float(Value) >0.3):
                Lister.append(7)
            elif (PayLoads2[1]/float(Value) <=0.3) & (PayLoads2[1]/float(Value) >0.2):
                Lister.append(8)
            elif (PayLoads2[1]/float(Value) <=0.2) & (PayLoads2[1]/float(Value) >0.1):
                Lister.append(9)
            elif (PayLoads2[1] <= 0.1):
                Lister.append(10)
        
    return (Key, Lister)


    
    


#def ConvertToSignificance(toyRDDLine):
    

#print(trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[1]))
    
YY = trainRDD.map(ValuesNonNumericFeatures).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).cache()

ZZ = trainRDD.map(NonNumericFeatures)\
             .reduceByKey(lambda x,y: x+y)\
             .map(ConvertForMerge)\
             .union(YY)\
             .reduceByKey(lambda x,y: [x] + [y])\
             .map(GetImportance).filter(lambda x: len(x[1]) != 0 ).cache()
TT = ZZ.collect()

ZZ.unpersist()
YY.unpersist()

#NamingDict = "HashDictionary" + str(Node)
from collections import defaultdict

NamingDict2 = defaultdict(list)

for ii in TT:
    NamingDict2[ii[0]] = ii[1]

In [ ]:
YYY.unpersist()

In [ ]:
Node = 6
YYY = sc.broadcast(NamingDict2)
def MappingChangesWithDictionary(trainRDD):
    
    
    Dictionary = YYY.value
    
    """ Taking in all the Key/Value Components """
    FinalKey = trainRDD[0]
    
    
    
    Value = trainRDD[1]
    Value[Node] = Dictionary[Value[Node]]
    
    return (FinalKey,Value[Node])


Hashing_6 = NonNumericRDD.map(MappingChangesWithDictionary).cache()
Hashing_6.sample(False, 0.0000001, None).collect()

In [ ]:
Hashing7 = Hashing_5.leftOuterJoin(Hashing_6).cache()
Hashing7.take(20)

In [ ]:
Hashing_5.unpersist()
Hashing_6.unpersist()